# 1. 生成模型

**生成模型**(Generative Model) 是一个广泛的机器学习领域，它是处理概率分布 $p(x)$ 的模型，其中 $x \in R_n$。这些模型是在某些潜在的高维空间中的数据点上定义的。通俗的理解，生成模型就是一类通过向真实数据分布学习，以此来确定其参数的模型，使得所学习的模型分布与真实数据分布尽可能地一致。但由于被建模随机变量的高维度，使得模型学习十分困难。

生成模型的工作流程：

1. 从一个已知分布（例如高斯分布或均匀分布）中随机采样样本；


2. 将该采样的样本输入到生成模型；


3. 生成模型根据真实分布反馈的信息不断更新自己的参数；


4. 经过若干次迭代，最后通过训练得到一个能够生成与真实数据尽可能一致的模型。

传统生成模型通常基于马尔可夫链、最大似然及近似推理，其代表模型有限制玻尔兹曼机(Restricted Boltzmann Machines, RBM)及其衍生模型如深度信念网络(Deep Belief Network， DBN)、深度波尔茨曼机(Deep Boltzmann Machines, DBM)、变分自动编码器(Variational Auto-Encoder, VAE)等，此类方法计算复杂且生成效果有限。

2014年， [Ian Goodfellow 等](https://arxiv.org/abs/1406.2661) 提出了基于博弈论的生成模型——**生成对抗网络**(Generative Adversarial Network, **GAN**)。该网络模型在生成图像数据方面的表现令人惊异，如今已经成为众多研究者的关注点。

除了图像合成，生成对抗网络在计算机视觉(CV)任务中还有很多的应用，如图像超分辨率、语义分割、图像编辑、图像修复、图像去噪、图像融合以及视频生成等。生成对抗网络在自然语言处理(NLP)中的应用也呈现日益增长的趋势， 例如：从文本生成图像、字体生成、对话生成、机器翻译、语音生成等。

# 2. 生成对抗网络(GAN)

## 2.1 GAN模型

受博弈论中二人零和博弈思想的启发，**GAN** 主要由**生成器**(Generator)和**判别器**(Discriminator)两个部分组成，**生成器**和**判别器**分别作为博弈的两方。其中，**生成器**的目的是生成接近真实的样本去骗过判别器， 而**判别器**是去区分真实的样本和生成的样本。通过对抗训练来不断的提高各自的能力，最终达到一个纳什均衡(Nash equilibrium)的状态。

- **生成器**$G$：它以一个随机向量（隐空间中的一个随机点）作为输入，并将其解码为一张生成图像。


- **判别器**$D$：以一张图像（真实的或生成的均可）作为输入，并预测该图像是来自训练集还是由生成器创建。

![img](images/chapter17/GAN.jpg)

> 对于生成器 $G$ 来说，通过随机噪声 $z$ 作为输入，生成器 $G$ 期望自己生成的样本尽可能地欺骗判别器 $D$，所以需要最大化判别概率 $D(G(z))$。因此，生成器 $G$ 的目标函数可以定义为最小化 $\log(1-D(G(z)))$。

$$ {\min_G} \; \mathbb{E}_{z\sim p_ {z}\;(z)}[\log(1-D(G(z)))] $$

> 对于判别器 $D$，为了尽可能地区分真实样本和虚假的生成样本，它希望最小化判别概率 $D(G(z))$ 的同时，最大化判别概率 $D(x)$。因此，判别器 $D$ 的目标函数可以定义为最大化 $ \log D(x) + \log(1-D(G(z)))$。
 
$$ {\max_D}\;\mathbb{E}_{x\sim p_ {\rm data}\;(x)}[\log D(x)] + \mathbb{E}_{z\sim p_ {z}\;(z)}[\log(1-D(G(z)))] $$

其中 $P_{\rm data}(x)$ 代表真实样本的概率分布，$P_{z}(z)$ 代表随机噪声的概率分布，$z$ 是服从高斯分布的随机噪声。

基于以上思想可以设计出 GAN 的总目标函数如下式所示：

$$ {\min_G} \; {\max_D}\; V(D, G) = \mathbb{E}_{x\sim p_ {\rm data}\;(x)}[\log D(x)] + \mathbb{E}_{z\sim p_ {\rm z}\;(z)}[\log(1-D(G(z)))] \tag{1} $$

对抗训练过程由两个神经网络交替进行：

- 先训练 $D$：选取一批真实样本和一批隐变量，隐变量通过 $G$ 得到生成样本，保持 $G$ 权值不变，利用随机梯度上升法计算并更新 $D$ 网络的权值；


- 随后训练 $G$：选取一批隐变量通过 $G$ 得到生成样本，保持 $D$ 权值不变，利用随机梯度下降法计算并更新 $G$ 网络的权值。

在对抗训练的过程中，判别器 $D$ 判别真假样本的能力逐渐提高；而生成器 $G$ 为了欺骗判别器 $D$，生成样本逐渐趋近于真实样本，最终使整个模型生成质量较好的新数据。

## 2.2 全局最优解

令 $p_g(x)$ 为 $G(z)$ 生成样本的概率分布， $p_{\rm data}(x)$ 为真实样本的概率分布，则式（1）可写成：

$$ \begin{aligned}
V(D, G) &= \mathbb{E}_{x\sim p_{\rm data}\;(x)}[\log D(x)] + \mathbb{E}_{x\sim p_{g}\;(x)}[\log (1-D(x))]\\
&= \int_x p_{\rm data}(x)\log (D(x))dx + \int_x p_g(x) \log(1-D(x))dx\\
&= \int_x p_{\rm data}(x)\log (D(x)) + p_g(x) \log(1-D(x))dx
\end{aligned} $$

对于固定 $G$，最大化函数 $V(D,G)$ 有

$$ \frac{\partial}{\partial D} \int_x p_{\rm data}(x)\log (D(x)) + p_g(x) \log(1-D(x))dx \triangleq 0 $$

$$ \Rightarrow \int_x p_{\rm data}(x) \frac{1}{D(x)} + p_g(x) \frac{-1}{1-D(x)}dx \triangleq 0 $$

$$ \Rightarrow p_{\rm data}(x) \frac{1}{D(x)} = p_g(x) \frac{1}{1-D(x)} $$

函数 $V(D,G)$ 达到极大值时的 $D(x) = D^*_G(x)$，即
$$ D^*_G(x) = \frac{p_{\rm data}(x)}{p_{\rm data}(x) + p_g(x)} \tag{2}$$

将 $D^*_G(x)$ 带入 $V(D, G)$ ：

$$ \begin{aligned}
{\min_G} \; {\max_D}\; V(D, G) &= {\min_G} \; V(D^*_G, G) \\
&= {\min_G} \; \mathbb{E}_{x\sim p_{\rm data}\;(x)}\left[\log \frac{p_{\rm data}(x)}{p_{\rm data}(x) + p_g(x)}\right ] + \mathbb{E}_{x\sim p_{g}\;(x)}\left[\log \frac{p_g(x)}{p_{\rm data}(x) + p_g(x)}\right]\\
&= {\min_G} \; \mathbb{E}_{x\sim p_{\rm data}\;(x)}\left[\log \frac{p_{\rm data}(x)}{(p_{\rm data}(x) + p_g(x))/2}\cdot \frac{1}{2}\right ] + \mathbb{E}_{x\sim p_{g}\;(x)}\left[\log \frac{p_g(x)}{(p_{\rm data}(x) + p_g(x))/ 2}\cdot \frac{1}{2}\right]\\
&= {\min_G} \; KL \left[p_{\rm data}(x) \bigg|\bigg| \frac{p_{\rm data}(x) + p_g(x)}{2}\right] + KL \left[p_g(x) \bigg|\bigg| \frac{p_{\rm data}(x) + p_g(x)}{2}\right] - \log 4
\end{aligned} \tag{3} $$

上式（3）中的 KL 散度值总是大于等于0，只有在两个分布相同 $p_g(x) = p_{\rm data}(x)$ 时，KL值等于0，此时上式取最小值 $-\log4$。同时，可计算式（2） $D^*_G(x) = \frac{1}{2}$，意味着 $D$ 至多以50%的概率随机猜测样本属于真实样本或生成样本。

## 2.3 生成结果

将采样的随机噪声通过多层感知机网络构建的生成器来生成手写体数字和人脸图像，如下图所示。最右边的一列显示了与相邻样本最近的训练样本，以证明生成模型没有记住训练集。

![GAN](images/chapter17/GAN_result.jpg)

## 2.4 小结

优点：

- 不要求对数据预设分布，直接进行采样和推断，非常适用于无监督和半监督学习任务；


- 各种类型的损失函数和约束条件都可以整合到 GAN 框架中，有利于针对不同任务设计出不同类型的损失函数和优化方法；


- 可以和现有的卷积神经网络(CNN) 、循环神经网络(RNN)等深度网络结合使用

缺点：


- 在模型训练过程中，存在**模式崩溃**问题：生成器从具有多种模式的训练集中仅学习到单个或有限的模式，而错失对其它模式的学习，从而无法生成具有多样性的样本；


- 使用随机向量作为生成器的输入缺乏语义和可操作性，无法控制模型生成具有指定特征的样本；


- 实际应用中比较难以训练，训练过程中易出现梯度消失、训练不稳定、神经网络难以收敛等问题

# 3. 改进的GAN

对 GAN 模型结构进行改进可以划分为输入输出、生成器、判别器、模型的模块结构和模块结构的组合思想 5 个部分：

# 4. [DCGAN(Deep Convolutional GAN)](https://arxiv.org/abs/1511.06434)

DCGAN 的生成器和判别器都使用了卷积神经网络(CNN)来替代原始 GAN 中的多层感知机，以此生成更高分辨率的图像。生成器模型如下图所示：

![DCGAN](images/chapter17/DCGAN.jpg)

DCGAN特点：

- 使用反卷积层（生成器）和带步长的卷积层（判别器）替换了池化层

- 在生成器和判别器中均使用了 BN 层

- 去掉了全连接层

- 使用 ReLU 作为生成器的激活函数（输出层使用 Tanh 激活函数）

- 使用 LeakyReLU 作为判别器的激活函数

![img](images/chapter17/DCGAN_result.jpg)

# 5. [pix2pix](https://arxiv.org/abs/1611.07004)

pix2pix 是将GAN应用于有监督的图像到图像翻译(image translation)的经典论文。该模型可以将输入图像作为条件，学习从输入图像到输出图像之间的映射，从而得到指定的输出图像。 ~~补充介绍风格迁移的其他应用（图像修补、老照片修复、图像去噪、卡通化）~~

![pix2pix](images/chapter17/pix2pix_result.jpg)

以训练轮廓图像到真实图像的生成为例，pix2pix 模型训练流程示意图如下图所示。轮廓图像 $x$ 作为生成器 $G$ 的输入（随机噪声 $z$ 在图中并未画出）得到生成图像 $G(x)$，然后将 $G(x)$ 和 $x$ 通道拼接作为判别器 $D$ 的输入。另外，将真实图像 $y$ 和 $x$ 也进行通道拼接作为判别器 $D$ 的输入。

![pix2pix](images/chapter17/pix2pix.jpg)

其中，生成器 $G$ 的网络模型可以采用传统的Encoder-decoder结构，或是采用在图像分割领域应用非常广泛的U-Net网络模型结构，而后者的生成效果更好。

![img](images/chapter17/pix2pix_G.jpg)

pix2pix 模型的优化目标分为两部分，分别为：

$$ \begin{aligned}
\mathcal L_{cGAN}(D, G) &= \mathbb{E}_{x,y}[\log D(x, y)] + \mathbb{E}_{x,z}[\log(1-D(x, G(x, z)))]\\
\mathcal L_{L1}(G) &= \mathbb{E}_{x,y,z}[||y-G(x,z)||_1]
\end{aligned}
$$

整体目标函数为：

$$ G^* = \arg {\min_G} \; {\max_D}\; V(D, G) = \mathcal L_{cGAN}(D, G) + \lambda \mathcal L_{L1}(G) $$

[pix2pix在线互动Demo](https://affinelayer.com/pixsrv/)

# 6. [CycleGAN](https://arxiv.org/abs/1703.10593)

CycleGAN 是一种无监督生成对抗网络，它的主要思想是训练两对生成器-判别器模型以将图像从一个域转换为另一个域。

![CycleGAN_result](images/chapter17/CycleGAN_result.jpg)

CycleGAN 模型如下图所示，由两个生成器 $G $ 、$F$ 和两个判别器 $D_Y$、$D_X$ 组成，其中 $G:X\rightarrow Y$，$F:Y\rightarrow X$

![CycleGAN_result](images/chapter17/CycleGAN.jpg)

模型的损失函数由两部分组成：

- Adversarial Loss，

$$ \mathcal L_{GAN}(G, D_Y, X, Y) = \mathbb{E}_{y\sim p_ {\rm data}\;(y)}[\log D_Y(y)] + \mathbb{E}_{x\sim p_ {\rm data}\;(x)}[\log(1-D_Y(G(x)))] $$
$$ \mathcal L_{GAN}(F, D_X, Y, X) = \mathbb{E}_{x\sim p_ {\rm data}\;(x)}[\log D_X(x)] + \mathbb{E}_{y\sim p_ {\rm data}\;(y)}[\log(1-D_X(F(y)))] $$

- Cycle Consistency Loss，目的是确保转换后的风格在反转换后可以回到原始状态

$$ \mathcal L_{cyc}(G, F) = \mathbb{E}_{x\sim p_ {\rm data}\;(x)}\left[||F(G(x))-x||_1 \right] + \mathbb{E}_{y\sim p_ {\rm data}\;(y)}\left[||G(F(y))-y||_1 \right] $$

因此总损失函数为：

$$ \mathcal L(G, F, D_X, D_Y) = \mathcal L_{GAN}(G, D_Y, X, Y) + \mathcal L_{GAN}(F, D_X, Y, X) + \lambda \mathcal L_{cyc}(G, F) $$

# 7. [WGAN(Wasserstein GAN)](https://arxiv.org/abs/1701.07875)

[Towards Principled Methods for Training Generative Adversarial Networks](https://arxiv.org/abs/1701.04862), [Improved Training of Wasserstein GANs](https://arxiv.org/abs/1704.00028)